In [ ]:
import cv2
import numpy as np
from matplotlib import pyplot as plt
import seaborn as sns
import pandas as pd
import os
import glob
import re
from itertools import groupby
from scipy import optimize
from scipy.signal import argrelextrema
from tqdm import tqdm

# Helper functions

In [ ]:
def rotate_and_crop_binary_image(binary_image):

   # Find contours in the binary image
   contours, _ = cv2.findContours(binary_image, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

   # Get the rotated bounding box
   contour = max(contours, key=cv2.contourArea)
   rect = cv2.minAreaRect(contour)
   box = cv2.boxPoints(rect)
   box = np.intp(box)

   # Compute the angle and size of the rotated bounding box
   # TODO: handle rotation of the box to ensure metacarpal head on LEFT side!
   size = (int(rect[1][0]), int(rect[1][1]))
   if size[0] < size[1]:
      # print('rotating')
      angle = rect[2] + 90
   else:
      angle = rect[2]

   # Rotate the image
   center = tuple(map(int, rect[0]))
   rotation_matrix = cv2.getRotationMatrix2D(center, angle, 1)
   rotated_image = cv2.warpAffine(binary_image, rotation_matrix, binary_image.shape[::-1], flags=cv2.INTER_LINEAR)

   # Crop the rotated image to the bounding box
   x, y, w, h = cv2.boundingRect(box)
   pad = 40
   cropped_image = rotated_image[y - pad : y + h + pad, x - pad : x + w + pad]
   # cropped_image = rotated_image[y : y + h, x: x + w]

   return cropped_image

In [ ]:
def get_articular_surface(mask):

    # get width of each point on the bone
    widths = []
    for x in range(mask.shape[1]):
        col = mask[:, x]
        if np.any(col):
            width = np.flatnonzero(col)[-1] - np.flatnonzero(col)[0]
            widths.append(width)
        else:
            widths.append(0)
    
    # find critical x value that maximizes thickness (the thickest point of bone)
    widths = np.array(widths)
    candidates = argrelextrema(widths, np.greater_equal)[0]
    critical_x = candidates[np.flatnonzero(widths[candidates])][0]

    # find the critical y_max and y_min (the widest part of shape)
    critical_y_max = np.flatnonzero(mask[:, critical_x])[0]
    critical_y_min = np.flatnonzero(mask[:, critical_x])[-1]

    # find contour of metacarpal head
    contours, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    cnt = max(contours, key=cv2.contourArea)
    coords = [(i[0][0], i[0][1]) for i in cnt]

    # find arc between top and bottom extrema
    arc = np.array([c for c in coords if c[0] <= critical_x])

    # calculate apex point
    apex_y = (critical_y_max + critical_y_min) // 2
    apex_x = np.flatnonzero(mask[apex_y, :])[0]

    return arc, (critical_x, critical_y_max), (critical_x, critical_y_min), (apex_x, apex_y)

In [ ]:
# calculate the distance of data points (x, y) from center (xc, yc)
def calc_R(xc, yc, x, y):
    return np.sqrt((x-xc)**2 + (y-yc)**2)

# calculate algebraic distance between data points (x, y) and the mean circle centered at (xc, yc)
def f(c, x, y):
    Ri = calc_R(*c, x, y)
    return Ri - Ri.mean()

In [ ]:
def circle_fitting(x, y):

    # coordinates of the barycenter
    x_m = np.mean(x)
    y_m = np.mean(y)

    # find best-fitting circle
    center_estimate = x_m, y_m
    center, ier = optimize.leastsq(f, center_estimate, args = (x, y))

    # record statistics for best-fitting circle
    Ri = calc_R(*center, x, y)
    R = Ri.mean()
    residual = sum((Ri - R)**2)

    return center, R, residual

In [ ]:
def visualize_geometry(cropped_img, topmost, bottommost, apex, center, a, R, accession, save=False, outdir=None):

    # visualize metacarpal head with associated geometry
    plt.imshow(cropped_img, cmap=plt.cm.gray)

    # line through endpoints of articular surface
    plt.plot([topmost[0], bottommost[0]], [topmost[1], bottommost[1]], color = 'red')

    # line showing radius of best-fit circle
    plt.plot([apex[0], apex[0]+ R], [apex[1], apex[1]], color = 'red')

    # all critical points
    [plt.scatter(*i, c='red', marker='o') for i in (topmost, bottommost)];
    [plt.scatter(*i, c='red', marker='|') for i in (apex, (apex[0] + R, apex[1]))];

    # show best-fitting circle
    circle = plt.Circle(center, R, fill=False, color='red')
    plt.gca().add_artist(circle)

    # add labels for text
    plt.text(0, -5, '\n'.join([f"a = {round(a, 2)}", 
        f"R = {round(R, 2)}", 
        f"ratio = {round(a / R, 4)}"]), 
        bbox=dict(facecolor='black', alpha=0.5),
        color='white')

    # title
    plt.title(accession)

    # show and save
    fig = plt.gcf()
    plt.show()
    plt.draw()
    if save: fig.savefig(os.path.join(outdir, f'{accession}.png'));

In [ ]:
def get_roundness(mask):

    # isolate metacarpal head
    cropped_img = rotate_and_crop_binary_image(mask)

    # get articular surface
    arc, bottommost, topmost, apex = get_articular_surface(cropped_img)

    # data points for articular surface
    x = arc[:, 0]
    y = arc[:, 1]

    # best-fit circle
    center, R, _ = circle_fitting(x, y)

    # calculate distance between endpoints of articular surface
    a = topmost[1] - bottommost[1]

    # calculate ratio
    roundness = a / R

    # save data
    args = (topmost, bottommost, apex, center, a, R)

    return roundness, cropped_img, args

# Sample one image

### Read images

In [ ]:
# UCL rupture

# mask_path = './UCL rupture - masked'
# all_mask_paths = [file for file in glob.glob(os.path.join(mask_path, '*')) if file.lower().endswith(('.jpg', '.jpeg', '.png'))]

# image_path = './UCL rupture - unmasked'
# all_image_paths = [file for file in glob.glob(os.path.join(image_path, '*')) if file.lower().endswith(('.jpg', '.jpeg', '.png'))]

# controls

# mask_path = './Controls - masked'
# all_mask_paths = [file for file in glob.glob(os.path.join(mask_path, '*')) if file.lower().endswith(('.jpg', '.jpeg', '.png'))]

# image_path = './Controls - unmasked'
# all_image_paths = [file for file in glob.glob(os.path.join(image_path, '*')) if file.lower().endswith(('.jpg', '.jpeg', '.png'))]

In [ ]:
# sample random scan
accession = re.search('[0-9]+', os.path.basename(np.random.choice(all_mask_paths))).group()

# retrieve image and mask (depends on directory structure)
img_sample_path = glob.glob(os.path.join(image_path, f'*{accession}*'))[0]
mask_sample_path = glob.glob(os.path.join(mask_path, f'*{accession}*'))[0]

img = cv2.imread(img_sample_path)
img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

mask = cv2.imread(mask_sample_path)
mask = cv2.cvtColor(mask, cv2.COLOR_BGR2GRAY)

nonzero_proportion = np.sum(mask) / (mask.shape[0] * mask.shape[1]) / 255
if nonzero_proportion == 0.0 or nonzero_proportion == 1.0:
    print('No mask found')
elif nonzero_proportion > 0.5:
    mask = cv2.bitwise_not(mask)

_, mask = cv2.threshold(mask, 127, 255, cv2.THRESH_BINARY)

plt.imshow(np.hstack([img, mask]), cmap=plt.cm.gray)

In [ ]:
# isolate metacarpal head and get extrema
cropped_img = rotate_and_crop_binary_image(mask)
plt.imshow(cropped_img, cmap=plt.cm.gray)

arc, bottommost, topmost, apex = get_articular_surface(cropped_img)
[plt.scatter(*i, c='red', marker='.') for i in arc];

In [ ]:
# data points for articular surface
x = arc[:, 0]
y = arc[:, 1]

# best-fit circle
center, R, _ = circle_fitting(x, y)

# calculate distance between endpoints of articular surface
a = topmost[1] - bottommost[1]

In [ ]:
# visualize
visualize_geometry(cropped_img, topmost, bottommost, apex, center, a, R, accession, save=False, outdir='./results_20230818/')

In [ ]:
print("a =", a)
print("R =", R)
print("ratio =", a / R)

# Calculate for entire dataset

In [ ]:
# loop through all masks and get roundness

df = pd.DataFrame(columns=['group', 'accession', 'roundness'])

control_path = './Controls - masked'
case_path = './UCL rupture - masked'

with open('./unfiltered_accessions.txt', 'w') as file:
    for is_control, dir_path in zip([True, False], [control_path, case_path]):

        count = 0
        error_count = 0
            
        # get all mask paths
        all_mask_paths = [file for file in glob.glob(os.path.join(dir_path, '*')) if file.lower().endswith(('.jpg', '.jpeg', '.png'))]

        for mask_path in tqdm(all_mask_paths):

            # get accession number
            accession = re.search('[0-9]+', os.path.basename(mask_path)).group()
            file.write(accession + '\n')
            count += 1

            # read mask
            mask = cv2.imread(mask_path)
            mask = cv2.cvtColor(mask, cv2.COLOR_BGR2GRAY)
            nonzero_proportion = np.sum(mask) / (mask.shape[0] * mask.shape[1]) / 255
            if nonzero_proportion == 0.0 or nonzero_proportion == 1.0:
                print('No mask found for accession:', accession)
                error_count += 1
                continue
            elif nonzero_proportion > 0.5:
                mask = cv2.bitwise_not(mask)
            _, mask = cv2.threshold(mask, 127, 255, cv2.THRESH_BINARY)

            # get roundness
            try:
                roundness, cropped_img, args = get_roundness(mask)
            except:
                print('Error in accession:', accession)
                error_count += 1
                continue

            # append to dataframe
            df = pd.concat([df, pd.DataFrame({'group': ['control' if is_control else 'case'], 'accession': [accession], 'roundness': [roundness]})], ignore_index=True)

            # save geometry
            visualize_geometry(cropped_img, *args, accession, save=True, outdir='./results_20231006/')

        print('Number of total accessions:', count)
        print('Number of errors:', error_count)

file.close()

In [ ]:
# save means and SEM for each group
control_mean = df[df['group'] == 'control'].roundness.mean()
control_sem = df[df['group'] == 'control'].roundness.std() / np.sqrt(len(df[df['group'] == 'control']))
case_mean = df[df['group'] == 'case'].roundness.mean()
case_sem = df[df['group'] == 'case'].roundness.std() / np.sqrt(len(df[df['group'] == 'case']))

# print
print('Control mean: ' + str(round(control_mean, 3)) + ' +/- ' + str(round(control_sem, 3)))
print('Case mean: ' + str(round(case_mean, 3)) + ' +/- ' + str(round(case_sem, 3)))

# Visualize distribution

In [ ]:
# plot
fig, ax = plt.subplots(figsize=(4, 4))
sns.barplot(x='group', y='roundness', data=df, ax=ax, palette='Set2')
ax.set(xlabel='', ylabel='roundness')
ax.set_xticklabels(['control', 'case'])
# ax.set_ylim(1, 2.5)
ax.set_title('Metacarpal roundness')
ax.text(0, 1.6, 'n = ' + str(len(df[df['group'] == 'control'])), ha='center')
ax.text(1, 1.6, 'n = ' + str(len(df[df['group'] == 'case'])), ha='center')
ax.text(0, 1.5, 'mean = ' + str(round(control_mean, 2)) + ' +/- ' + str(round(control_sem, 2)), ha='center')
ax.text(1, 1.5, 'mean = ' + str(round(case_mean, 2)) + ' +/- ' + str(round(case_sem, 2)), ha='center')
fig.tight_layout()

In [ ]:
# plot
fig, ax = plt.subplots(figsize=(4, 4))
sns.stripplot(x='group', y='roundness', data=df, ax=ax, palette='Set2')
ax.set(xlabel='', ylabel='roundness')
ax.set_xticklabels(['control', 'case'])
# ax.set_ylim(1, 2.5)
ax.set_title('Metacarpal roundness')
ax.text(0, 2.5, 'n = ' + str(len(df[df['group'] == 'control'])), ha='center')
ax.text(1, 2.5, 'n = ' + str(len(df[df['group'] == 'case'])), ha='center')
ax.text(0, 2.4, 'mean = ' + str(round(control_mean, 2)) + ' +/- ' + str(round(control_sem, 2)), ha='center')
ax.text(1, 2.4, 'mean = ' + str(round(case_mean, 2)) + ' +/- ' + str(round(case_sem, 2)), ha='center')
fig.tight_layout()

In [ ]:
# plot histogram of roundness for control and case groups

df[df['group'] == 'control'].roundness.hist(bins=15, alpha=0.5, label='control', color='#66c2a5', density=True)
sns.kdeplot(df[df['group'] == 'control'].roundness, color='#66c2a5', alpha=1, linestyle='--')

df[df['group'] == 'case'].roundness.hist(bins=15, alpha=0.5, label='case', color='#fc8d62', density=True)
sns.kdeplot(df[df['group'] == 'case'].roundness, color='#fc8d62', alpha=1, linestyle='--')

plt.xlabel('roundness')
plt.ylabel('density')
plt.legend()

In [ ]:
# sort and examine range of roundness values
df.sort_values(by=['roundness'], inplace=False)

# Statistical testing

In [ ]:
# split into case/control
controls = df[df['group'] == 'control']['roundness']
cases = df[df['group'] == 'case']['roundness']

In [ ]:
from scipy.stats import ttest_ind
result = ttest_ind(controls, cases, alternative='two-sided')
result